<a href="https://colab.research.google.com/github/melihkurtaran/MachineLearning/blob/main/Supervised_Learning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Supervised Learning Project**

In this project, a dataset which has been collected using readings of a multi-spectral imaging sensor installed in a drone intended
to map a specific geographical area will be used for developing supervised machine learning models

In [1]:
#Load libraries
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean, std
from scipy.stats import sem
from matplotlib import pyplot
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
import random
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.model_selection import cross_validate

In [2]:
#Connect to GitHub for faster access
!git clone https://github.com/melihkurtaran/MachineLearning.git

Cloning into 'MachineLearning'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 20 (delta 5), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.


In [3]:
# CSV to DataFrame 
ds_09 = pd.read_csv("MachineLearning/SupervisedLearning/ds_09.csv")
ds_09.head()

,V01,V02,V03,V04,V05,V06,V07,V08,V09,V10,...,V29,V30,V31,V32,V33,V34,V35,V36,class,target
0,14.65,NaN,46.46,15.74,49.84,17.95,21.59,60.14,2.85,8.99,...,59.10,13.22,0.00,11.56,3.70,17.89,13.76,15.75,4,0.5272
1,13.97,NaN,19.38,5.99,21.44,31.61,45.76,26.47,2.52,26.66,...,32.93,12.02,14.30,17.82,3.15,11.77,14.46,16.38,1,0.4937
2,12.14,53.27,62.25,11.42,28.51,33.03,42.41,52.27,4.68,25.59,...,25.71,12.50,23.18,14.60,4.08,5.87,6.52,14.25,2,0.5796
3,8.29,16.06,15.41,6.97,14.81,16.53,29.76,36.73,3.14,20.58,...,31.74,9.67,8.43,21.87,6.03,10.20,12.54,7.82,5,0.4098
4,10.02,47.28,45.67,10.43,13.07,NaN,35.22,19.96,3.34,35.50,...,NaN,12.20,18.88,23.72,3.16,15.35,8.81,14.93,1,0.5465


There are 36 features and 2 values to be used in classification and regression tasks

# **T1 - Dataset preparation**

The dataset needs to be preprocessed before using in models

##**(a) removing missing values and outliers**

Samples have 38 rows, class and target row are never missing so thresh needs to setas 34 to drop samples with more than 4 missing feature values.

In [4]:
# samples with more than 4 missing feature values are dropped
print("Size before dropping: " + str(len(ds_09)))
ds_09 = ds_09.dropna(axis=0, thresh=34) # thresh: Require that many non-NA values
print("Size after dropping: " + str(len(ds_09)))

Size before dropping: 1000
Size after dropping: 967


Filling remaining null values with the mean

In [5]:
# the remaining missing values are filled using the average value

for i in ds_09.columns[ds_09.isnull().any(axis=0)]:  #Applying only on variables with NaN valuesfor bettter performance
    ds_09[i].fillna(ds_09[i].mean(),inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [6]:
# We can see that we do not have any missing values anymore
ds_09.isnull().values.any()

False

Removing outliers

In [7]:
# samples with at least one feature value with a z-score higher than 3 (i.e. an outlier) are discarded
print("Size before removing outlier samples: " + str(len(ds_09)))
# code below for each column, first calculates Z-score of each value in the column, and remove all rows that have outliers in at least one column
ds_09 = ds_09[(np.abs(stats.zscore(ds_09)) < 3).all(axis=1)] # axis=1 ensures that for each row, all column satisfy the constraint.
print("Size after removing outlier samples: " + str(len(ds_09)))

Size before removing outlier samples: 967
Size after removing outlier samples: 869


##**(b) Dimensionality Reduction**

Keep only features that account for up to 95% of the variance of the data

In [8]:
#correlation matrix function with sckit library 

##**(c) Standardization**

mu-sigma standardization is used to normalize the features

In [9]:
ds_09.reset_index(inplace = True)
X = ds_09.copy()
X.drop(['class', 'target'], axis=1, inplace=True)
y = ds_09['class']

In [10]:
# define mu-sigma standardizer scaler
ss = StandardScaler()
  
# transform data
X = pd.DataFrame(ss.fit_transform(X),columns = X.columns)
X.head()

,index,V01,V02,V03,V04,V05,V06,V07,V08,V09,...,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36
0,-1.740876,1.214996,-0.054054,1.152872,2.357207,2.216477,-1.496285,-1.158382,2.376606,-0.895622,...,-0.316122,-1.539320,2.279143,0.406997,-3.207810,-0.729132,0.408947,0.383839,1.212194,-0.099109
1,-1.737393,0.986969,-0.054054,-1.939001,-1.175792,-0.726378,-0.047792,1.256262,-0.882371,-1.204081,...,-1.312862,0.261232,-0.129392,0.010242,-0.254762,0.577851,-0.075171,-0.882510,1.432997,0.053858
2,-1.733911,0.373306,0.726206,2.955703,0.791817,0.006227,0.102783,0.921588,1.614855,0.814919,...,1.431467,-1.003106,-0.793879,0.168944,1.579018,-0.094431,0.743429,-2.103338,-1.071547,-0.463317
3,-1.730428,-0.917733,-2.285514,-2.392278,-0.820680,-1.413390,-1.646860,-0.342179,0.110712,-0.624553,...,-1.802451,-0.385048,-0.238912,-0.766735,-1.466957,1.423423,2.459851,-1.207375,0.827364,-2.024555
4,-1.726946,-0.337604,0.241384,1.062673,0.433082,-1.593692,-0.044691,0.203289,-1.512486,-0.437608,...,1.481963,-0.760398,0.015906,0.069755,0.691039,1.809672,-0.066369,-0.141737,-0.349203,-0.298209


##**(d) Calculate IR**

The Imbalance Ratio (IR) is the ratio between the number of samples from the majority class and the number of samples from the minority class

In [11]:
ds_09['class'].value_counts() #Observe majority and minority class

1    230
4    224
5    148
0    132
3     68
2     67
Name: class, dtype: int64

In [12]:
IR = ds_09['class'].value_counts().max() / ds_09['class'].value_counts().min()
print('Imbalance Ratio: ' + str(IR))

Imbalance Ratio: 3.4328358208955225


# **T2 - Classifier design (I)**

4 different classifiers are defined in this task

## **(a) Find the best configuration for each model**

Best parameters found by GridSearchCV

Since the data imbalanced, it is better to use F1 score not the accuracy so all optimization is done based on weighted F1 score

### **1) Gaussian Naive Bayes (GNB)**

In [13]:
from sklearn.naive_bayes import GaussianNB
model_gnb = GaussianNB()

### **2) Logistic Regression (LR)**

In [14]:
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(solver='saga', max_iter=5000, tol=0.001)
param_lr = [{'penalty': ['l1','l2','none'],'C': [0.1, 1, 10]}]

gs_lr = GridSearchCV(model_lr,
                      param_grid=param_lr,
                      scoring='f1_weighted',
                      cv=5)
gs_lr.fit(X, y)
gs_lr.best_params_

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the

{'C': 0.1, 'penalty': 'l2'}

Best C value is 0.1 and best penalty value is l2.

In [15]:
model_lr = LogisticRegression(solver='saga', max_iter=5000, tol=0.001, C=0.1, penalty='l2')

### **3) Decision Tree (DT)**

In [16]:
from sklearn import tree
model_dt = tree.DecisionTreeClassifier()
param_dt = [{'criterion': ['gini','entropy'],'max_depth': [3, 5, None],'min_samples_leaf': [1, 5, 10]}]

gs_dt = GridSearchCV(model_dt,
                      param_grid=param_dt,
                      scoring='f1_weighted',
                      cv=5)
gs_dt.fit(X, y)
gs_dt.best_params_

{'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1}

Best criterion value is gini, better to set max_dept default which is None and best min. sample leaf is 5.

In [17]:
model_dt = tree.DecisionTreeClassifier(criterion='gini', max_depth=None, min_samples_leaf=5)

### **4) Support Vector Machine (SVM)**

In [18]:
from sklearn import svm
model_svm = svm.SVC(gamma='scale', probability=True, max_iter=5000, tol=0.05)
param_svm = [{'kernel': ['linear','poly','rbf'],'C': [0.1, 1, 10]}]

gs_svm = GridSearchCV(model_svm,
                      param_grid=param_svm,
                      scoring='f1_weighted',
                      cv=5)
gs_svm.fit(X, y)
gs_svm.best_params_

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-

{'C': 10, 'kernel': 'rbf'}

Best C value is 10 and the best kernel value is rbf

In [19]:
model_svm = svm.SVC(gamma='scale', probability=True, max_iter=5000, tol=0.05, C=10, kernel='rbf')

## **b) Estimate the accuracy, precision, recall and F1 scores**

In [20]:
from sklearn.metrics import make_scorer
scoring = {'acc': 'accuracy',
           'pre': make_scorer(precision_score, average='weighted'),
           'rec': make_scorer(recall_score, average='weighted'),
           'f1': make_scorer(f1_score, average='weighted')}
scores = cross_validate(model_lr, X, y, scoring=scoring,
                        cv=5, return_train_score=True)
scores

{'fit_time': array([0.0318799 , 0.02947497, 0.02919888, 0.02931404, 0.02943206]),
 'score_time': array([0.00462127, 0.00453043, 0.00445962, 0.00444031, 0.00444078]),
 'test_acc': array([0.56321839, 0.61494253, 0.63218391, 0.60344828, 0.58381503]),
 'train_acc': array([0.65611511, 0.65611511, 0.65467626, 0.64460432, 0.6566092 ]),
 'test_pre': array([0.593314  , 0.63839611, 0.64943386, 0.62475765, 0.59685491]),
 'train_pre': array([0.66810536, 0.67560125, 0.67427411, 0.66132729, 0.6752897 ]),
 'test_rec': array([0.56321839, 0.61494253, 0.63218391, 0.60344828, 0.58381503]),
 'train_rec': array([0.65611511, 0.65611511, 0.65467626, 0.64460432, 0.6566092 ]),
 'test_f1': array([0.5659557 , 0.61686259, 0.62403712, 0.60223362, 0.57727596]),
 'train_f1': array([0.65557257, 0.65689527, 0.65591898, 0.64502982, 0.65789364])}

# **T3 - Classifier design (II)**

# **T4 - Regression**

# **T5 - Model exploitation**

In [21]:
df_t5 = pd.read_csv('MachineLearning/SupervisedLearning/im_x_09.txt', sep=" ", header=None, index_col=False)
df_t5_y = pd.read_csv('MachineLearning/SupervisedLearning/im_y_09.txt', sep=" ", header=None, index_col=False)
df_t5_t = pd.read_csv('MachineLearning/SupervisedLearning/im_t_09.txt', sep=" ", header=None, index_col=False)
df_t5['class'] = df_t5_y
df_t5['target'] = df_t5_t

df_t5.head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,class,target
0,10.56,53.90,28.49,9.56,35.65,38.17,45.36,22.96,3.43,27.41,...,27.15,15.54,18.56,19.07,2.82,14.04,14.95,19.99,0,0.557017
1,11.94,34.57,45.58,15.47,39.31,31.83,32.69,49.95,2.83,39.21,...,48.16,13.69,7.20,17.95,3.33,6.18,10.37,13.16,0,0.594420
2,6.29,45.65,31.93,11.38,39.51,36.15,29.35,37.18,5.32,20.21,...,32.70,16.90,16.10,12.59,2.09,11.36,14.21,17.10,0,0.519508
3,10.34,52.78,26.24,12.49,29.88,39.67,38.91,36.10,2.06,35.34,...,26.78,13.29,13.92,15.35,2.51,15.46,10.57,16.88,0,0.536709
4,8.25,53.67,30.91,3.57,18.04,27.73,28.09,19.02,3.41,31.42,...,34.27,12.07,15.83,23.11,2.11,18.25,6.78,9.40,0,0.510596
